In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from scipy.io import FortranFile
from cobaya.model import get_model
import yaml
import os
from getdist import plots, MCSamples
from cobaya.conventions import Const, packages_path_input


In [ ]:
path_GLM_data = os.path.realpath(os.path.join(os.getcwd(), 'GLM_data'))

In [ ]:
getdist_settings = {
    'smooth_scale_1D': 0.3,
    'smooth_scale_2D': 0.4,
    'boundary_correction_order': 1,
    'mult_bias_correction_order': 1,
    }

## ACT

In [ ]:
yaml_path = os.path.realpath(os.path.join(os.getcwd(), 'chains/01p3_LCDM_CMB_ACTTTTEEE.input.yaml'))

In [ ]:
# define the path to the YAML parameters file:
yaml_params = yaml_path #os.path.join(chains_dir, name + '.input.yaml')
# load the parameters from the YAML file:
with open(yaml_params, 'r') as f:
    params_dict = yaml.safe_load(f)
# remove the 'path' key from the 'camb' dictionary if it exists:
if 'path' in params_dict.get('theory', {}).get('camb', {}):
    del params_dict['theory']['camb']['path']
if 'recombination_model' in params_dict.get('theory', {}).get('camb', {}).get('extra_args', {}):
    del params_dict['theory']['camb']['extra_args']['recombination_model']
if 'max_zrei'in params_dict.get('theory', {}).get('camb', {}).get('extra_args', {}):
    del params_dict['theory']['camb']['extra_args']['max_zrei']
    print('Removed "max_zrei" from the "camb" extra_args in the YAML parameters file.')

# build the model:
model = get_model(params_dict)

In [ ]:
model.components[0].__dict__

In [ ]:
ACT_datavector = model.components[0].data_vec

In [ ]:
ACT_covmat = model.components[0].covmat

In [ ]:
ACT_corr = ACT_covmat / np.outer( np.sqrt(np.diag(ACT_covmat)), np.sqrt(np.diag(ACT_covmat)))

plt.imshow(ACT_corr)

In [ ]:
chains_dir = os.path.join('chains')

# import chain
from cobaya import load_samples

name = '01p3_LCDM_CMB_ACTTTTEEE'
chain = load_samples(os.path.join(chains_dir, name), skip=0.3, to_getdist=True)
chain.label = name
chain.root = os.path.join(chains_dir, name)
chain.updateSettings(getdist_settings)
chain.updateBaseStatistics()
chain.getConvergeTests()
print(chain.label+': R =', round(chain.GelmanRubin, 4))

In [ ]:
params = model.parameterization.sampled_params()
print(params)

In [ ]:
BestFit = chain.getBestFit()
BestFitDict = BestFit.getParamDict()

In [ ]:
for p in params.keys():
    params[p] = BestFitDict[p]  # Set the best fit values for all parameters

In [ ]:
model.provider.set_current_input_params(params)
logposterior = model.loglike(params, return_derived=False)
Cls = model.provider.get_Cl(ell_factor=True)

In [ ]:
Cls

In [ ]:
model.components[0].__dict__

In [ ]:
model.components[0].spec_meta[0]

In [ ]:
lmin, lmax = 600, 6500

ls = Cls['ell']

mask_ls = (ls >= lmin) & (ls <= lmax)

In [ ]:
ps_vec = np.zeros_like(ACT_datavector)

for m in model.components[0].spec_meta:
    idx = m["idx"]
    win = m["window"].weight.T
    ls = m["window"].values
    pol = m["pol"]
    dat = Cls[pol][ls] / (params['A_act'] * params['A_act'])
    if pol[0] == "e":
        dat /= params['P_act']
    if pol[1] == "e":
        dat /= params['P_act']

    ps_vec[idx] = win @ dat

In [ ]:
plt.plot(ACT_datavector)
plt.plot(ps_vec)

## Planck High-L

In [ ]:
yaml_path = os.path.realpath(os.path.join(os.getcwd(), 'chains/01p2_LCDM_CMB_PlanckTTTEEE.minimize.input.yaml'))

# define the path to the YAML parameters file:
yaml_params = yaml_path #os.path.join(chains_dir, name + '.input.yaml')
# load the parameters from the YAML file:
with open(yaml_params, 'r') as f:
    params_dict = yaml.safe_load(f)
# remove the 'path' key from the 'camb' dictionary if it exists:
if 'path' in params_dict.get('theory', {}).get('camb', {}):
    del params_dict['theory']['camb']['path']
if 'recombination_model' in params_dict.get('theory', {}).get('camb', {}).get('extra_args', {}):
    del params_dict['theory']['camb']['extra_args']['recombination_model']
if 'max_zrei'in params_dict.get('theory', {}).get('camb', {}).get('extra_args', {}):
    del params_dict['theory']['camb']['extra_args']['max_zrei']
    print('Removed "max_zrei" from the "camb" extra_args in the YAML parameters file.')
# build the model:
model = get_model(params_dict)

In [ ]:
Planck_hl_cov = model.components[0].cov
Planck_hl_corr = Planck_hl_cov / np.outer( np.sqrt(np.diag(Planck_hl_cov)),  np.sqrt(np.diag(Planck_hl_cov)))

plt.imshow(Planck_hl_corr)

In [ ]:
chains_dir = os.path.join('chains')

# import chain
from cobaya import load_samples

name = '01p2_LCDM_CMB_PlanckTTTEEE'
chain = load_samples(os.path.join(chains_dir, name), skip=0.3, to_getdist=True)
chain.label = name
chain.root = os.path.join(chains_dir, name)
chain.updateSettings(getdist_settings)
chain.updateBaseStatistics()
chain.getConvergeTests()
print(chain.label+': R =', round(chain.GelmanRubin, 4))

In [ ]:
params = model.parameterization.sampled_params()
print(params)

BestFit = chain.getBestFit()
BestFitDict = BestFit.getParamDict()


for p in params.keys():
    params[p] = BestFitDict[p]  # Set the best fit values for all parameters


model.provider.set_current_input_params(params)
logposterior = model.loglike(params, return_derived=False)
Cls = model.provider.get_Cl(ell_factor=True)

In [ ]:
model.components[0].__dict__

In [ ]:
blmax = model.components[0].blmax
blmin = model.components[0].blmin

used_bins = model.components[0].used_bins
weights = model.components[0].weights

used_indices = model.components[0].used_indices


In [ ]:
def binning_matrix(ix=0):
    # not used by main likelihood code
    lmax = blmax[used_bins[ix][-1]]
    lmin = blmin[used_bins[ix][0]]
    m = np.zeros((len(used_bins[ix]), lmax - lmin + 1))
    for i in used_bins[ix]:
        m[i, blmin[i] - lmin:blmax[i] + 1 - lmin] = weights[
                                                                blmin[i]:
                                                                blmax[i] + 1]
    return lmin, lmax, m

In [ ]:
L0 = Cls['ell'][0]

cl = np.empty(used_indices.shape)
ix = 0
for tp, cell in enumerate([Cls['tt'], Cls['te'], Cls['ee']]):
    for i in used_bins[tp]:
        cl[ix] = np.dot(cell[blmin[i] - L0:blmax[i] - L0 + 1],
                        weights[blmin[i]:blmax[i] + 1])
        ix += 1
cl /= params['A_planck'] ** 2

In [ ]:
Planck_datavector = model.components[0].X_data

In [ ]:
plt.loglog(Planck_datavector)
plt.loglog(cl)

# BAO

In [ ]:
yaml_path = os.path.realpath(os.path.join(os.getcwd(), 'chains/01p1_LCDM_BAO_DESIDR2.input.yaml'))

# define the path to the YAML parameters file:
yaml_params = yaml_path #os.path.join(chains_dir, name + '.input.yaml')
# load the parameters from the YAML file:
with open(yaml_params, 'r') as f:
    params_dict = yaml.safe_load(f)
# remove the 'path' key from the 'camb' dictionary if it exists:
if 'path' in params_dict.get('theory', {}).get('camb', {}):
    del params_dict['theory']['camb']['path']
if 'recombination_model' in params_dict.get('theory', {}).get('camb', {}).get('extra_args', {}):
    del params_dict['theory']['camb']['extra_args']['recombination_model']
if 'max_zrei'in params_dict.get('theory', {}).get('camb', {}).get('extra_args', {}):
    del params_dict['theory']['camb']['extra_args']['max_zrei']
    print('Removed "max_zrei" from the "camb" extra_args in the YAML parameters file.')
# build the model:
model = get_model(params_dict)

In [ ]:
model.provider?

In [ ]:
cov = model.components[0].cov
corr = cov / np.outer(np.sqrt(np.diag(cov)), np.sqrt(np.diag(cov)))
plt.imshow(corr)

In [ ]:
np.save(path_GLM_data + "cov_data_BAO.npy", cov)

In [ ]:
redshifts = np.array(model.components[0].data['z'])
BAO_datavector = np.array(model.components[0].data['value'])
observables = np.array(model.components[0].data['observable']).astype(str)

In [ ]:
chains_dir = os.path.join('chains')

# import chain
from cobaya import load_samples

name = '01p1_LCDM_BAO_DESIDR2'
chain = load_samples(os.path.join(chains_dir, name), skip=0.3, to_getdist=True)
chain.label = name
chain.root = os.path.join(chains_dir, name)
chain.updateSettings(getdist_settings)
chain.updateBaseStatistics()
chain.getConvergeTests()
print(chain.label+': R =', round(chain.GelmanRubin, 4))

In [ ]:
params = model.parameterization.sampled_params()
print(params)

BestFit = chain.getBestFit()
BestFitDict = BestFit.getParamDict()


for p in params.keys():
    params[p] = BestFitDict[p]  # Set the best fit values for all parameters


model.provider.set_current_input_params(params)
logposterior = model.loglike(params, return_derived=False)

rdrag = model.components[0].provider.get_param("rdrag")
rs = rdrag * model.components[0].rs_rescale

In [ ]:
def theory_fun(z, observable):
    # Functions to get the corresponding theoretical prediction:
    # Spherically-averaged distance, over sound horizon radius
    if observable == "DV_over_rs":
        return np.cbrt(
            ((1 + z) * model.provider.get_angular_diameter_distance(z)) ** 2 *
            Const.c_km_s * z / model.provider.get_Hubble(z,
                                                        units="km/s/Mpc")) / rs
    # Idem, inverse
    elif observable == "rs_over_DV":
        return np.cbrt(
            ((1 + z) * model.provider.get_angular_diameter_distance(z)) ** 2 *
            Const.c_km_s * z / model.provider.get_Hubble(z, units="km/s/Mpc")
        ) ** (-1) * rs
    # Comoving angular diameter distance, over sound horizon radius
    elif observable == "DM_over_rs":
        return (1 + z) * model.provider.get_angular_diameter_distance(z) / rs
    # Physical angular diameter distance, over sound horizon radius
    elif observable == "DA_over_rs":
        return model.provider.get_angular_diameter_distance(z) / rs
    # Hubble distance [c/H(z)] over sound horizon radius.
    elif observable == "DH_over_rs":
        return 1 / model.provider.get_Hubble(z, units="1/Mpc") / rs
    # Hubble parameter, times sound horizon radius
    elif observable == "Hz_rs":
        return model.provider.get_Hubble(z, units="km/s/Mpc") * rs
    # Diff Linear Growth Rate times present amplitude
    elif observable == "f_sigma8":
        return model.provider.get_fsigma8(z)
    # Anisotropy (Alcock-Paczynski) parameter
    elif observable == "F_AP":
        return ((1 + z) * model.provider.get_angular_diameter_distance(z) *
                model.provider.get_Hubble(z, units="km/s/Mpc")) / Const.c_km_s

In [ ]:
BAO_model = np.array([theory_fun(redshifts[i], observables[i]) for i in range(len(redshifts))]).flatten()

In [ ]:
plt.plot(BAO_model)
plt.plot(BAO_datavector)

# DES Supernovae

In [ ]:
yaml_path = os.path.realpath(os.path.join(os.getcwd(), 'chains/01p5_LCDM_SN_DESY5.input.yaml'))

# define the path to the YAML parameters file:
yaml_params = yaml_path #os.path.join(chains_dir, name + '.input.yaml')
# load the parameters from the YAML file:
with open(yaml_params, 'r') as f:
    params_dict = yaml.safe_load(f)
# remove the 'path' key from the 'camb' dictionary if it exists:
if 'path' in params_dict.get('theory', {}).get('camb', {}):
    del params_dict['theory']['camb']['path']
if 'recombination_model' in params_dict.get('theory', {}).get('camb', {}).get('extra_args', {}):
    del params_dict['theory']['camb']['extra_args']['recombination_model']
if 'max_zrei'in params_dict.get('theory', {}).get('camb', {}).get('extra_args', {}):
    del params_dict['theory']['camb']['extra_args']['max_zrei']
    print('Removed "max_zrei" from the "camb" extra_args in the YAML parameters file.')
# build the model:
model = get_model(params_dict)

In [ ]:
cov = model.components[0].covs['mag']
corr = cov / np.outer(np.sqrt(np.diag(cov)), np.sqrt(np.diag(cov)))
plt.imshow(corr)

In [ ]:
cols = model.components[0].cols

mag = model.components[0].mag 
invcov = model.components[0].invcov

zcmb = model.components[0].zcmb
zhel = model.components[0].zhel

In [ ]:
chains_dir = os.path.join('chains')

# import chain
from cobaya import load_samples

name = '01p5_LCDM_SN_DESY5'
chain = load_samples(os.path.join(chains_dir, name), skip=0.3, to_getdist=True)
chain.label = name
chain.root = os.path.join(chains_dir, name)
chain.updateSettings(getdist_settings)
chain.updateBaseStatistics()
chain.getConvergeTests()
print(chain.label+': R =', round(chain.GelmanRubin, 4))

In [ ]:
params = model.parameterization.sampled_params()
print(params)

BestFit = chain.getBestFit()
BestFitDict = BestFit.getParamDict()


for p in params.keys():
    params[p] = BestFitDict[p]  # Set the best fit values for all parameters


model.provider.set_current_input_params(params)
logposterior = model.loglike(params, return_derived=False)


In [ ]:
angular_diameter_distances = model.provider.get_angular_diameter_distance(zcmb)
SN_model = 5 * np.log10((1 + zhel) * (1 + zcmb) * angular_diameter_distances) 


In [ ]:
plt.plot(SN_model + 25, label='model')
plt.plot(mag, label='data')
plt.title('DESY5 Supernovae')
plt.xlabel("SN #")
plt.ylabel("Mag")
plt.legend()

In [ ]:
model.components[0].__dict__

In [ ]:
model.components[0].__dict__